In [316]:
%matplotlib inline

In [318]:
#parallel
from ipyparallel import Client
rc = Client(profile="default")
dview = rc[:] # use all engines
lv = rc.load_balanced_view()

In [319]:
%%px --local
import os,  ast
import numpy as np
import pandas as pd
import vcf, pyfasta

chromosomes = ["CAE" + str(i) for i in range(1,30) + ["X"] + ['Y'] ]
eu = os.path.expanduser
jn = os.path.join
var_ana_dir = eu("~/vervet_project/analyses/20150504_163_UnifiedGenotyper/_data")
meta_dir = eu("~/vervet_project/metadata")

def str_to_dic(s):
    try:
        return ast.literal_eval(s)
    except ValueError:
        print s
        raise
        
mt = pd.read_csv(jn(meta_dir,"163_master_table.csv"),
                 index_col=0,
                 converters={"library":str_to_dic, "phenotypes":str_to_dic})


#gene_df_ensembl = pd.read_csv(jn(meta_dir,"annot_ensemble_1.1.77_genes.tsv"),sep="\t",index_col=[0,1])
#gene_df_ensembl_CDS = pd.read_csv(jn(meta_dir,"annot_ensemble_1.1.77_CDS.tsv"),sep="\t",index_col=[0,1])
mac_seq = pyfasta.Fasta(eu("~/vervet_project/data/other_species/Macaca_mulatta/CR_1.0_in_vervet_coord_ref3500.fa"))
#gene_df_r100 = pd.read_csv(jn(meta_dir,"annot_release100_genes.tsv"),sep="\t",index_col=[0,1])
#gene_df_r100_exons = pd.read_csv(jn(meta_dir,"annot_release100_exons.tsv"),sep="\t",index_col=[0,1])
#attention, upgrade to ensembl 1.1.78!!!
#gene_df_ensembl = pd.read_csv(jn(meta_dir,"annot_ensemble_1.1.77_genes.tsv"),sep="\t",index_col=[0,1])

In [320]:
%%px --local

import re
import subprocess, gzip

def add_info_line(info_dic, line):
    #split all commas except if in quotes
    r = re.compile('[^,]+".+"[^,]*'
                '|'
                '[^,]+')
    try:
        category, value = line.strip().split('=<')
    except ValueError:
        try: 
            info_dic["uncategorised"].update({line.strip().split('=')[0][2:]:line.strip().split('=')[1]})
        except KeyError:
            try:
                info_dic.update({"uncategorised":{line.strip().split('=')[0][2:]:line.strip().split('=')[1]}})
            except ValueError, e:
                print line.strip().split('=')
                raise e
        return
    category = category[2:]
    tags = r.findall(value[:-1])
    subdic = {k:v for k,v in [t.split('=',1) for t  in tags]}
    try:
        info_dic[category].update({subdic["ID"]:subdic})
    except KeyError:
        info_dic.update({category:{subdic["ID"]:subdic}})
        
def parse_vcf_header(fn):
    info_dic = {}
    fh = gzip.open(fn) if fn[-3:] == ".gz" else open(fn)
    for line in fh:
        if line[:6] == "#CHROM":
            header = line.strip().split('\t')
            header[0] = header[0][1:]
        else:
            add_info_line(info_dic, line)
        if line[0] != '#':
            break
    return header, info_dic

In [321]:
%%px --local
def get_vcf_df(fn, chrom, start, end, header=None, **kwa):
    if header is None:
        header, _ = parse_vcf_header(fn)
    tabix_stream = subprocess.Popen(['tabix',fn,"{}:{}-{}".format(chrom,start,end)], stdout=subprocess.PIPE,stderr=subprocess.PIPE)
    vcf_df = pd.read_csv(tabix_stream.stdout, sep="\t", index_col=[0,1], header=False, names = header, **kwa)
    
    return vcf_df

## Get SNPeff annotations for peak regions, including filtered sites and including macacaque differences

In [18]:
peaks = [("CAE1",2467224),("CAE1",2757224),("CAE1",7005224),
         ("CAE2",50381341),("CAE4",5189544),("CAE5",13975063),
        ("CAE5",60269063),("CAE5",74650063), ("CAE6",3823905),
        ("CAE6",5564905),("CAE6",11705905),("CAE6",29586905),
        ("CAE8",117925766),("CAE10",104389305),
        ("CAE14",17153077),("CAE14",98746077),("CAE15",62257003),
        ("CAE16",1907271),("CAE16",6963271),("CAE16",7241271),("CAE16",57481271),
        ("CAE16",59802271),("CAE16",68026271),("CAE17",44458229),("CAE17",54486229),
        ("CAE18",62028538),("CAE20",18933658),("CAE20",106492658),("CAE20",111605658),
        ("CAE22",10431400),("CAE23",250753),("CAE23",42363534),("CAE24",3453536),
        ("CAE24",8944536),("CAE25",61650817),("CAE26",18910982),("CAE29",18370875),
        ("CAE29",21874875)]

In [322]:
%%px --local
def get_genotype_df(chrom, start,end):
    vcf_fn = jn(var_ana_dir,"163_UG_ref3500_all_sites_{}_filter_anc_mac_diff_ensembl_1.1.78.vcf.gz".format(chrom))


    header, info_dic = parse_vcf_header(vcf_fn)
    samples = header[9:]

    converters = {"INFO":get_info_dic}
    converters.update({n:get_genotype for n in samples})#{i:get_genotype for t in range(9,len(header))}

    vcf_df = get_vcf_df(vcf_fn, chrom, start, end, header=header, converters=converters)
    
    return vcf_df

In [323]:
%%px --local
def get_genotype(s):
    gt_str = s.split(':',1)[0]
    if gt_str[:3] in ["0/0","0|0"]:
        return 0
    elif gt_str[:3] in ["1/1","1|1"]:
        return 2
    elif gt_str[:3] in ["0/1","0|1","1|0"]:
        return 1
    else:
        return np.nan
    
def get_info_dic(s):
    info_tuples = [t.split('=') for t in s.split(';')]
    info_tuples = [t for t in info_tuples if len(t)==2]
    tags = [t for t in info_tuples if len(t)!=2]
#    try:
    d = {k:v for (k,v) in info_tuples}
    d.update({'_tags':tags})
#    except ValueError:
#        d = {}
#        logging.warning("Can not parse info column at {}:{}".format(line[0],line[1]))
    return d

def group_by_pop(col,poptype="pop3"):
    return mt.ix[col][poptype]

In [324]:
%%px --local
def try_dic(dic,key):
    try:
        return dic[key]
    except KeyError:
        return np.nan

def get_ith_annotation(el,i):
    try:
        return el.split(',')[i]
    except IndexError:
        return np.nan
def split_annotation(s):
    return pd.Series(s.split('|')[:4]+s.split('|')[-1:],index=['allele','effect','impact','gene_symbol','errors'])

In [341]:
%%px --local
def get_variant_annot_df(vcf_df, sample_ids=None):
    #remove non-seggregating
    
    if sample_ids is None:
        sample_ids = vcf_df.columns.values[7:]
    vcf_df = vcf_df[vcf_df['ALT'] != '.']

    subpop_af_df = vcf_df[sample_ids].groupby(group_by_pop,axis=1).apply(lambda df: df.mean(axis=1)/2.)
    annot_df = vcf_df[["REF","ALT","FILTER"]]
    annot_df.loc[:,'AA'] = vcf_df['INFO'].apply(lambda d:try_dic(d,"AA"))
    annot_df.loc[:,'AF'] = vcf_df[sample_ids].mean(axis=1)/2.

    annot_s = vcf_df['INFO'].apply(lambda d:try_dic(d,"ANN"))



    #annotation lowest level
    level0_df = annot_s.apply(lambda el:get_ith_annotation(el,0)).apply(split_annotation)
    level0_df = pd.concat([annot_df,level0_df,subpop_af_df],axis=1)
    level0_df.set_index("gene_symbol",append=True,inplace=True)
    tot_level_df = level0_df
    i =1
    for i in range(1,100):
        level_info = annot_s.apply(lambda el:get_ith_annotation(el,i)).dropna()   
        if not len(level_info):
            break
        level_df = level_info.apply(split_annotation)
        level_df = annot_df.ix[level_df.index].join(level_df,how='left').join(subpop_af_df, how='left')
        level_df.set_index("gene_symbol",append=True,inplace=True)
        tot_level_df = pd.concat([tot_level_df, level_df])
    return tot_level_df

In [348]:
%%px --local
def get_annot(chrom, pos, distance):
    start = pos-distance
    end = pos+distance
    try:
        vcf_df = get_genotype_df(chrom, start,end)
        annot_df = get_variant_annot_df(vcf_df)
    except Exception, e:
        return str(e)
    return annot_df

In [349]:
peak_map = lv.map_async(lambda p:get_annot(p[0],p[1],50000),peaks)

In [351]:
peak_map.wait(10)

In [353]:
[err for err in peak_map.result if type(err)==str]

['Too many columns specified: expected 172 and found -1']

In [355]:
annot_f = pd.concat([r for r in peak_map.result if type(r)!=str])

In [361]:
annot_f[annot_f['impact']=='MODERATE'].xs('RANBP3',level=2)

REF ALT FILTER   AA        AF allele            effect  \
CHROM POS                                                              
CAE6  5576627   G   A   PASS  NaN  0.000000      A  missense_variant   
      5576646   C   T  ExCov  NaN  0.081250      T  missense_variant   
      5576672   G   A  ExCov  NaN  0.000000      A  missense_variant   
      5576717   G   A  ExCov  NaN  0.000000      A  missense_variant   
      5586838   A   G   PASS  NaN  0.015924      G  missense_variant   

                 impact errors       aet      cyn  pyn  pys  sab  sac  sar  \
CHROM POS                                                                    
CAE6  5576627  MODERATE         0.000000  0.00000    0    0    0    0    0   
      5576646  MODERATE         0.766667  0.09375    0    0    0    0    0   
      5576672  MODERATE         0.000000  0.00000    0    0    0    0    0   
      5576717  MODERATE         0.000000  0.00000    0    0    0    0    0   
      5586838  MODERATE         0.000000  0.15625    0    0    0    0    0   

               tan  
CHROM POS           
CAE6  5576627    0  
      5576646    0  
      5576672    0  
      5576717    0  
      5586838    0

In [362]:
annot_f[annot_f['impact']=='MODERATE'].xs('NFIX',level=2)

,,REF,ALT,FILTER,AA,AF,allele,effect,impact,errors,aet,cyn,pyn,pys,sab,sac,sar,tan
CHROM,POS,,,,,,,,,,,,,,,,,
CAE6,11707284,T,G,PASS,NaN,0,G,missense_variant,MODERATE,,0,0,0,0,0,0,0,0


In [ ]:
annot_f[annot_f['impact']=='MODERATE'].xs('NFIX',level=2)

In [309]:
define a metric for af differentiation....

REF   ALT                         FILTER AA        AF  \
CHROM POS                                                                 
CAE16 6955661       T     C                          ExCov  T  0.215190   
      6955663       G     A                 5bpIndel;ExCov  G  0.035948   
      6955667       T   TAG                 5bpIndel;ExCov  T  0.613924   
      6955676       G     C                          ExCov  C  0.000000   
      6955704       T     C                          ExCov  C  0.706250   
      6955828       C     G                       5bpIndel  G  0.000000   
      6955832       T   TCC                       5bpIndel  T  0.009375   
      6955865       C     T                           PASS  C  0.003106   
      6956049       T     C                          ExCov  C  0.000000   
      6956068       G     T                          ExCov  G  0.560127   
      6956089      AG     A  5bpIndel;ExCov;QualityByDepth  A  0.018868   
      6956108       C     T   ExCov;LowQual;QualityByDepth  C  0.016026   
      6956109       A     G   ExCov;LowQual;QualityByDepth  A  0.016129   
      6956128       A    AT  5bpIndel;ExCov;QualityByDepth  A  0.036184   
      6956129       T     G  5bpIndel;ExCov;QualityByDepth  T  0.019481   
      6956242       C    CG                 5bpIndel;ExCov  C  0.020000   
      6956243       C   A,G                 5bpIndel;ExCov  C  0.011719   
      6956294       G     A                          ExCov  G  0.006329   
      6956384       C     T                  ExCov;LowQual  C  0.003205   
      6956404       G     A                          ExCov  G  0.215686   
      6956431       G     C                          ExCov  G  0.013245   
      6956546       G     C                       5bpIndel  G  0.036424   
      6956548       G  GCTT                       5bpIndel  G  0.029801   
      6956569      CT     C                       5bpIndel  C  0.019481   
      6956573       C     G                       5bpIndel  C  0.417219   
      6956606       C     T                           PASS  C  0.019231   
      6956615       A     G                           PASS  G  0.593333   
      6956663       A     G                           PASS  A  0.024845   
      6956676       G     C                           PASS  C  0.000000   
      6956677       G     A                           PASS  A  0.000000   
...               ...   ...                            ... ..       ...   
      6958637       G     A                           PASS  A  0.000000   
      6958672       G     T               5bpIndel;LowQual  T  0.000000   
      6958672       G     T               5bpIndel;LowQual  T  0.000000   
      6958672       G     T               5bpIndel;LowQual  T  0.000000   
      6958672       G     T               5bpIndel;LowQual  T  0.000000   
      6958672       G    GC                       5bpIndel  T  0.006536   
      6958672       G    GC                       5bpIndel  T  0.006536   
      6958672       G    GC                       5bpIndel  T  0.006536   
      6958672       G    GC                       5bpIndel  T  0.006536   
      6958672       G     T               5bpIndel;LowQual  T  0.000000   
      6958672       G     T               5bpIndel;LowQual  T  0.000000   
      6958672       G     T               5bpIndel;LowQual  T  0.000000   
      6958672       G     T               5bpIndel;LowQual  T  0.000000   
      6958672       G    GC                       5bpIndel  T  0.006536   
      6958672       G    GC                       5bpIndel  T  0.006536   
      6958672       G    GC                       5bpIndel  T  0.006536   
      6958672       G    GC                       5bpIndel  T  0.006536   
      6958686  AACCCC     A                       5bpIndel  A  0.003226   
      6958703       C     T                           PASS  C  0.519481   
      6958758       C     T                           PASS  T  0.000000   
      6958770       C     T                

In [7]:
from pypopgen import vcfparser

In [310]:
tot_level_df[tot_level_df["impact"]=='MODERATE'].xs("CD68",level=2)

REF ALT         FILTER AA        AF allele            effect  \
CHROM POS                                                                    
CAE16 6957216   G   A          ExCov  G  0.003205      A  missense_variant   
      6957221   G   A          ExCov  G  0.009615      A  missense_variant   
      6957354   G   A          ExCov  G  0.056962      A  missense_variant   
      6957485   A   G          ExCov  A  0.022727      G  missense_variant   
      6957509   G   A          ExCov  A  0.000000      A  missense_variant   
      6957557   A   G          ExCov  G  0.000000      G  missense_variant   
      6957623   C   T          ExCov  C  0.022876      T  missense_variant   
      6957677   A   G  ExCov;LowQual  G  0.000000      G  missense_variant   
      6957762   T   C          ExCov  T  0.097484      C  missense_variant   
      6957774   C   T          ExCov  T  0.000000      T  missense_variant   
      6957835   G   A  ExCov;LowQual  G  0.003185      A  missense_variant   
      6958256   T   C          ExCov  C  0.000000      C  missense_variant   
      6958279   G   C          ExCov  G  0.276730      C  missense_variant   

                 impact errors      aet       cyn  pyn       pys       sab  \
CHROM POS                                                                    
CAE16 6957216  MODERATE         0.03125  0.000000    0  0.000000  0.000000   
      6957221  MODERATE         0.00000  0.000000    0  0.030000  0.000000   
      6957354  MODERATE         0.56250  0.000000    0  0.000000  0.000000   
      6957485  MODERATE         0.21875  0.000000    0  0.000000  0.000000   
      6957509  MODERATE         0.00000  0.000000    0  0.000000  0.000000   
      6957557  MODERATE         0.00000  0.000000    0  0.000000  0.000000   
      6957623  MODERATE         0.00000  0.033333    0  0.058824  0.000000   
      6957677  MODERATE         0.00000  0.000000    0  0.000000  0.000000   
      6957762  MODERATE         0.00000  0.000000    0  0.303922  0.000000   
      6957774  MODERATE         0.00000  0.000000    0  0.000000  0.000000   
      6957835  MODERATE         0.00000  0.000000    0  0.000000  0.000000   
      6958256  MODERATE         0.00000  0.000000    0  0.000000  0.000000   
      6958279  MODERATE         0.00000  0.000000    0  0.000000  0.708333   

                    sac  sar       tan  
CHROM POS                               
CAE16 6957216  0.000000  0.0  0.000000  
      6957221  0.000000  0.0  0.000000  
      6957354  0.000000  0.0  0.000000  
      6957485  0.000000  0.0  0.000000  
      6957509  0.000000  0.0  0.000000  
      6957557  0.000000  0.0  0.000000  
      6957623  0.000000  0.0  0.000000  
      6957677  0.000000  0.0  0.000000  
      6957762  0.000000  0.0  0.000000  
      6957774  0.000000  0.0  0.000000  
      6957835  0.000000  0.0  0.045455  
      6958256  0.000000  0.0  0.000000  
      6958279  0.758065  0.7  0.000000

In [29]:
class SNPEffAnnotationAlleleFreq(vcfparser.Parser):
    def __init__(self):
        self.annot_df = pd.DataFrame() 
    
    def parse_fun(self,sline):
        output = self._filter_(sline)
        if output is not None:
            self.out_tsv.write("\t".join(output)+"\n")

In [37]:
chrom = "CAE1"

In [38]:
variant = open(jn(var_ana_dir,"163_UG_ref3500_all_sites_{}_filter_anc_mac_diff_ensembl_1.1.78.vcf.gz".format(chrom)))

In [40]:
t = tabix.open(variant.name)

In [45]:
peaks[1]-distance

TypeError: unsupported operand type(s) for -: 'tuple' and 'int'

In [46]:
region = t.query(peaks[0][0],peaks[0][1]-distance,peaks[0][1]+distance)

In [48]:
dir(region)

['__class__',
 '__delattr__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__iter__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'next']

In [44]:
distance = 50000

In [32]:
distance = 50000 #bp to left and right
#chrom, pos = peaks[0]

intervals = [(chrom,pos-distance,pos+distance) for chrom,pos in peaks][:1]

snp_eff_parser = SNPEffAnnotationAlleleFreq(variant=)

variant = open(jn(var_ana_dir,"163_UG_ref3500_all_sites_{}_filter_anc_mac_diff_ensembl_1.1.78.vcf.gz".format(chrom)))
walker = vcfparser.get_walker(avariant, snp_eff_parser, intervals=intervals, chunk=False, skip_multiple_entries=False)

In [ ]:
parser_class = parser_classes[args.parser]
#    logging.warning("INTERVAL INPUT: {}".format(args.intervals))
    parser = parser_class(**{arg:getattr(sub_args,arg) for arg in vars(sub_args)})


    walker = get_walker(args.variant, parser, intervals=args.intervals,
                                            auto_tabix=args.auto_tabix,
                            chunk=not args.no_chunk, chunk_factor=args.chunk_factor,
                                 tmp_dir=args.temp_dir, ncpus=args.ncpus,
                           skip_multiple_entries=args.skip_multiple_entries,
                            progress_report_interval=args.progress_report_interval)

In [20]:

gene_df_ensembl.

In [24]:
gene_df_ensembl

end             gene_id           gene_name  \
chrom start                                                          
CAE1  3829            5154  ENSCSAG00000011340  ENSCSAG00000011340   
      7313           10166  ENSCSAG00000011331                ODF3   
      11325          11434  ENSCSAG00000022912  ENSCSAG00000022912   
      15562          17569  ENSCSAG00000011317               BET1L   
      19013          25473  ENSCSAG00000011307               RIC8A   
      19573          19643  ENSCSAG00000020592  ENSCSAG00000020592   
      26927          48944  ENSCSAG00000011291               SIRT3   
      49558          66974  ENSCSAG00000011249              PSMD13   
      93334          98972  ENSCSAG00000011172               NLRP6   
      102136        108370  ENSCSAG00000011141               ATHL1   
      112469        113448  ENSCSAG00000011109              IFITM5   
      128138        129734  ENSCSAG00000011104  ENSCSAG00000011104   
      147184        148057  ENSCSAG00000011099  ENSCSAG00000011099   
      194757        206519  ENSCSAG00000011092            B4GALNT4   
      215874        224015  ENSCSAG00000011081                PKP3   
      224726        228976  ENSCSAG00000011075              SIGIRR   
      235702        249990  ENSCSAG00000011063                ANO9   
      255089        255396  ENSCSAG00000021823         Metazoa_SRP   
      265246        303824  ENSCSAG00000011045              PTDSS2   
      305282        321270  ENSCSAG00000011038                RNH1   
      343506        346637  ENSCSAG00000011033                HRAS   
      355982        369633  ENSCSAG00000011029              LRRC56   
      370338        374345  ENSCSAG00000011007              LMNTD2   
      376405        379640  ENSCSAG00000010983              RASSF7   
      383379        383489  ENSCSAG00000027014  ENSCSAG00000027014   
      397520        429668  ENSCSAG00000010968               PHRF1   
      429310        432925  ENSCSAG00000010883                IRF7   
      435622        443156  ENSCSAG00000010823               CDHR5   
      444601        445339  ENSCSAG00000010809                 SCT   
      455700        459482  ENSCSAG00000010793                DRD4   
...                    ...                 ...                 ...   
CAEX  129315465  129342193  ENSCSAG00000006563              FUNDC2   
      129349054  129351326  ENSCSAG00000006553                CMC4   
      129352805  129353322  ENSCSAG00000006547               MTCP1   
      129358862  129403352  ENSCSAG00000006538  ENSCSAG00000006538   
      129487616  129509313  ENSCSAG00000006524  ENSCSAG00000006524   
      129536347  129542075  ENSCSAG00000006507              RAB39B   
      129560658  129606438  ENSCSAG00000006505               CLIC2   
      129626286  129729592  ENSCSAG00000006499               TMLHE   
      129808874  129809416  ENSCSAG00000000481  ENSCSAG00000000481   
      129838996  129839862  ENSCSAG00000000518               SPRY3   
      129937046  129984119  ENSCSAG00000006492               VAMP7   
      130006899  130020894  ENSCSAG00000006485                IL9R   
CAEY  3899           32376  ENSCSAG00000018322  ENSCSAG00000018322   
      99000         128638  ENSCSAG00000018321  ENSCSAG00000018321   
      373229        400632  ENSCSAG00000018320  ENSCSAG00000018320   
      489840        626098  ENSCSAG00000018318  ENSCSAG00000018318   
      689380        706092  ENSCSAG00000018317  ENSCSAG00000018317   
      968810       1188253  ENSCSAG00000018316  ENSCSAG00000018316   
      2512762      2531915  ENSCSAG00000018315  ENSCSAG00000018315   
      2626940      2668114  ENSCSAG00000018314  ENSCSAG00000018314   
      2638256      2638312  ENSCSAG00000020832  ENSCSAG00000020832   
      2885467      2902561  ENSCSAG00000018313  ENSCSAG00000018313   
      3016251      3048094  ENSCSAG00000018312  ENSCSAG00000018312   
      3938440      3938501  ENSCSAG00000021637                  U7   
      4017611      4021871  ENSCS

## For all genes: get for "pass" variants number of syn, nonsyn, high impact and allele frequencies in each category in all pops